In [ ]:
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import sys

# Input

In [ ]:
# Data folders
folderDATA = '...' # change ... by user input (change \ to /)
folderCalib = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/Camera_calibration/Matrix'
folderImages = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/4_points_of_view'
folderScripts = f'{folderDATA}/DATA_THESIS_MARTEN/Scripts'
folderVis = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/3D_reconstruct_checkerboard'

sys.path.insert(0, folderScripts)
# from datImage import * (datImage in reconstruct)
from reconstruct import *

# Camera info
pixel_width = 6.12e-3 # [mm] infosheet PathoViewer

# Reconstruct

In [ ]:
# Find pinholes and projections
pinholes, projections = find_pinholes_projections(
    filesImages = sorted(glob.glob(f'{folderImages}/CLR*.DAT')),
    fileMtx = f'{folderCalib}/mtx.npy',
    fileDist = f'{folderCalib}/dist.npy',
    pixel_width = pixel_width)
print(pinholes)
print(projections.shape)

In [ ]:
# Try one reconstruction
p = 21
print()
print(pinholes)
print(projections[:, p, :])

obj_estimation, obj_estimations = find_images(pinholes = pinholes, projected = projections[:, p, :], visualize=False)
print()
print(obj_estimation)

In [ ]:
# Try many reconstructions
reconstructions = []
all_obj_estimations = []

for p in range(projections.shape[1]):
    obj_estimation, obj_estimations = find_images(pinholes = pinholes, projected = projections[:, p, :], visualize=False)
    reconstructions += [obj_estimation]
    all_obj_estimations += [obj_estimations]
reconstructions = np.array(reconstructions)

In [ ]:
all_obj_estimations

# Visualize

In [ ]:
# RGB plots
def extract_filename(file_path):
    parts = file_path.replace("\\", "/").split("/")
    filename = parts[-1]
    return filename.split(".")[0]

filesImages = sorted(glob.glob(f'{folderImages}/CLR*.DAT'))
for f in filesImages:
    img = rescale(read_RGB(f), percentiles = None, to_uint8=True)
    fname = extract_filename(f)
    print(fname)
    plt.figure()
    plt.imshow(img)
    plt.savefig(f'{folderVis}/{fname}.jpg')
    plt.show()

In [ ]:
elev = 30

for azim in np.linspace(0,90,4):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    ax.scatter(reconstructions[:,0], reconstructions[:,1], reconstructions[:,2])
    ax.view_init(elev=elev, azim=azim)
    plt.savefig(f'{folderVis}/reco-{elev}-{azim}.jpg')
    plt.show()

In [ ]:
elev = 0
azim = 0

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter(reconstructions[:,0], reconstructions[:,1], reconstructions[:,2])
ax.view_init(elev=elev, azim=azim)
plt.savefig(f'{folderVis}/reco-{elev}-{azim}.jpg')

plt.show()

In [ ]:
elev = 0
azim = 90


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter(reconstructions[:,0], reconstructions[:,1], reconstructions[:,2])
ax.view_init(elev=elev, azim=azim)
plt.savefig(f'{folderVis}/reco-{elev}-{azim}.jpg')

plt.show()

# Checks

In [ ]:
from skspatial.objects import Plane, Line, Points

In [ ]:
CHECKERBOARD = (6,8)

In [ ]:
# Coplanarity

points = Points(reconstructions)
best_fit_plane = Plane.best_fit(points)
distances = [best_fit_plane.distance_point(point) for point in points]
# error = np.sqrt(np.dot(distances, distances) / len(distances))
mean_dist = np.mean(distances)
print('mean_dist:', mean_dist)
print('max_dist:', np.max(distances))

plt.figure()
plt.hist(distances)
plt.show()

In [ ]:
# Collinearity

def plot_selection(reconstructions, selection):
    elev = 0
    azim = 90
    roll = 0
    fig = plt.figure(figsize=(8,10))
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    ax.scatter(reconstructions[:,0], reconstructions[:,1], reconstructions[:,2], color = 'lightblue')
    ax.scatter(selection[:,0], selection[:,1], selection[:,2], color = 'red', marker = 's', s = 100)
    ax.view_init(elev=elev, azim=azim, roll=roll)
    plt.show()    


N = len(reconstructions)
all_distances = []
    
for i in range(CHECKERBOARD[1]):
    selection = reconstructions[i*CHECKERBOARD[0]:(i+1)*CHECKERBOARD[0]]
    points = Points(selection)
    best_fit_line = Line.best_fit(points)
    distances = [best_fit_line.distance_point(point) for point in points]
    all_distances += distances
#     print(len(all_distances))

for i in range(CHECKERBOARD[0]):
    selection = reconstructions[np.arange(i, N, CHECKERBOARD[0])]
    points = Points(selection)
    best_fit_line = Line.best_fit(points)
    distances = [best_fit_line.distance_point(point) for point in points]
    all_distances += distances
#     print(len(all_distances))

    
mean_dist = np.mean(all_distances)
print('mean_dist:', mean_dist)
print('max_dist:', np.max(distances))

plt.figure()
plt.hist(distances)
plt.show()

In [ ]:
# Orthogonality (upper right corners)
selections = []
cosines = []
for i in range(CHECKERBOARD[1]-1):
    for j in range(CHECKERBOARD[0]-1):
        selection = reconstructions[(i*CHECKERBOARD[0]+j, i*CHECKERBOARD[0]+j+1, i*CHECKERBOARD[0]+j+CHECKERBOARD[0]), :]
#         plot_selection(reconstructions, selection)
        u = selection[1]-selection[0]
        v = selection[2]-selection[0]
        
        selections += [selection]
        cosines += [np.dot(u,v)/np.linalg.norm(u)/np.linalg.norm(v)]
        
cosines=np.array(cosines)

print('Mean absolute difference from 90 deg:', 180/np.pi*np.mean(np.abs(np.pi/2-np.arccos(cosines))), 'deg')
print('Max absolute difference from 90 deg:', 180/np.pi*np.max(np.abs(np.pi/2-np.arccos(cosines))), 'deg')


plt.figure()
plt.hist(180/np.pi*(np.pi/2-np.arccos(cosines)))
plt.show()

In [ ]:
cosines_normalized = (cosines-np.min(cosines))/(np.max(cosines)-np.min(cosines))

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter(reconstructions[:,0], reconstructions[:,1], reconstructions[:,2])

for i in range(len(selections)):
    plt.plot(
        [selections[i][1,0], selections[i][0,0], selections[i][2,0]],
        [selections[i][1,1], selections[i][0,1], selections[i][2,1]],
        [selections[i][1,2], selections[i][0,2], selections[i][2,2]],
        color = (cosines_normalized[i], 0, 1-cosines_normalized[i])
    )

ax.view_init()
plt.show()



In [ ]:
# Edge lengths (should be 6.12 mm)

edge_lengths_x = []
edge_lengths_y = []


for i in range(CHECKERBOARD[1]):
    for j in range(CHECKERBOARD[0]-1):
        selection = reconstructions[(i*CHECKERBOARD[0]+j, i*CHECKERBOARD[0]+j+1), :]
#         plot_selection(reconstructions, selection)
        edge_lengths_x += [np.linalg.norm(selection[0] - selection[1])]
        
for i in range(CHECKERBOARD[1]-1):
    for j in range(CHECKERBOARD[0]):
        selection = reconstructions[(i*CHECKERBOARD[0]+j, i*CHECKERBOARD[0]+j+CHECKERBOARD[0]), :]
#         plot_selection(reconstructions, selection)
        edge_lengths_y += [np.linalg.norm(selection[0] - selection[1])]
        
edge_lengths_x = np.array(edge_lengths_x)        
edge_lengths_y = np.array(edge_lengths_y)    
edge_lengths = np.concatenate([edge_lengths_x, edge_lengths_y])
        

plt.figure(figsize=(20,8))
plt.subplot(121)
plt.hist(edge_lengths_x)
plt.title('Edge lengths x')
plt.subplot(122)
plt.hist(edge_lengths_y)
plt.title('Edge lengths y')
plt.show()

In [ ]:
edge_length_true = 6.14
print('Mean x deviation:', np.mean(np.abs(edge_length_true-edge_lengths_x)), 'mm')
print('Max x deviation:', np.max(np.abs(edge_length_true-edge_lengths_x)), 'mm')

print()
print('Mean y deviation:', np.mean(np.abs(edge_length_true-edge_lengths_y)), 'mm')
print('Max y deviation:', np.max(np.abs(edge_length_true-edge_lengths_y)), 'mm')

print()
print('Mean deviation:', np.mean(np.abs(edge_length_true-edge_lengths)), 'mm')
print('Max deviation:', np.max(np.abs(edge_length_true-edge_lengths)), 'mm')

In [ ]:
# Distances between non-averaged reconstructions
distances = np.array([np.linalg.norm(all_obj_estimations[i] - reconstructions[i], axis = 1)
 for i in range(len(reconstructions))])

print('mean_dist:', np.mean(distances))
print('max_dist:', np.max(distances))

plt.figure()
plt.hist(distances.flatten())
plt.show()

print(np.mean(distances, axis = 0))

plt.figure()
plt.hist(distances, bins = 100)
plt.legend([f'cam {i}' for i in range(distances.shape[1])])
plt.show()

In [ ]:

elev = 90
roll = 0
azim = 0

for i in range(len(reconstructions)):
    fig = plt.figure(figsize=(15,6))
    
    ax = fig.add_subplot(1, 3, 1, projection='3d')
    ax.scatter(all_obj_estimations[i][:,0], all_obj_estimations[i][:,1], all_obj_estimations[i][:,2])
    ax.scatter(reconstructions[i,0], reconstructions[i,1], reconstructions[i,2])
    ax.view_init(elev=90, azim=0, roll=0)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')

    ax = fig.add_subplot(1, 3, 2, projection='3d')
    ax.scatter(all_obj_estimations[i][:,0], all_obj_estimations[i][:,1], all_obj_estimations[i][:,2])
    ax.scatter(reconstructions[i,0], reconstructions[i,1], reconstructions[i,2])
    ax.view_init(elev=0, azim=90, roll=0)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    
    ax = fig.add_subplot(1, 3, 3, projection='3d')
    ax.scatter(all_obj_estimations[i][:,0], all_obj_estimations[i][:,1], all_obj_estimations[i][:,2])
    ax.scatter(reconstructions[i,0], reconstructions[i,1], reconstructions[i,2])
    ax.view_init(elev=0, azim=0, roll=0)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    
    plt.show()